결론 : 기사제목 크롤링으로는 우리가 원하고자하는 데이터들을 얻기가 힘들다고 판단  
→ 검색 횟수를 기반으로 하여 선호도를 알아보기로 결정 (검색량은 용현님이 크롤링)



In [ ]:
# 내 애플리케이션 인증
client_id = "Bx57kkdcEBHtn_M0QSjD"
client_secret = "j4qSysOfuR"

In [ ]:
import os
import sys
import urllib.request
import requests
import json
import pandas as pd

from bs4 import  BeautifulSoup
from datetime import datetime
import re
import os

# API 요청할때 요청헤더에 client_id와 client_secret 추가해야함
client_id = "Bx57kkdcEBHtn_M0QSjD"
client_secret = "j4qSysOfuR"

date = str(datetime.now()) 
date = date[:date.rfind(':')].replace(' ', '_') 
date = date.replace(':','시') + '분' 

query = input('검색 키워드를 입력하세요 : ') 
query = query.replace(' ', '+') 

news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))

news_url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}"

req = requests.get(news_url.format(query))
soup =  BeautifulSoup(req.text, 'html.parser')

news_dict = {} 
idx = 0 
cur_page = 1

while idx < news_num:
# 네이버 뉴스 웹페이지 구성이 바뀌어 태그명, class 속성 값 등을 수정   
  table = soup.find('ul',{'class' : 'list_news'})
  li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
  area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
  a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]
  
  for n in a_list[:min(len(a_list), news_num-idx)]:
      news_dict[idx] = {'title' : n.get('title'),
                        'url' : n.get('href') }
      idx += 1

  cur_page += 1
  
  pages = soup.find('div', {'class' : 'sc_page_inner'})
  next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')
  
  req = requests.get('https://search.naver.com/search.naver' + next_page_url)
  soup = BeautifulSoup(req.text, 'html.parser')

# df로 변환
  news_df = pd.DataFrame(news_dict).T

검색 키워드를 입력하세요 : 여름 회
총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : 10


In [ ]:
news_df.head(30)

,title,url
0,식중독은 여름에만? 겨울철 '노로바이러스' 주의,https://kormedi.com/?p=1380766
1,[입맛 돋는 여행 - 울릉도] 나리분지 산나물과 싱싱한 수산물의 보고 산채비빔밥과 물회,http://www.ktsketch.co.kr/news/articleView.htm...
2,회 뜨는데만 2시간…'1m 대물' 방어로 끓인 김치찌개 맛은 [백종원의사계MDI],https://www.joongang.co.kr/article/25045716
3,"태안 물회국수, ""매년 여름이면 여관 한 채 살 수 있을 정도"" 인기 비결은?",http://www.slist.kr/news/articleView.html?idxn...
4,"[A 유통가 소식] 현대백화점, 2022 봄·여름 해외패션대전 진행 外",http://www.asiaa.co.kr/news/articleView.html?i...
5,여름바다 맛 품은 한그릇 '가진항물회',http://www.mygoyang.com/news/articleView.html?...
6,"'2TV 생생정보' 30년 전통 제철 모둠회, 문어장·매운탕…여름별미",http://www.celuvmedia.com/article.php?aid=1628...
7,[전라도 味스토리] 여름 이 맛 ④장흥 삭금 된장물회,http://www.mdilbo.com/detail/SuoXpc/650699
8,"한치물회, 부드럽고 쫄깃한 식감…남녀노소 ""좋아요""",https://www.news1.kr/articles/?4455219
9,"'생방송 투데이' 가자미 물회 국수 맛집, 여름 별미로 인기 '침샘 자극'",http://www.celuvmedia.com/article.php?aid=1626...
